In [67]:
import numpy as np
from tqdm import tqdm_notebook

In [78]:
def check_strat(total_len_sampled, n_iters = 100):
    contain = 0
    
    for _ in tqdm_notebook(range(n_iters)):
    
        general_strata_1_size = 10**7
        general_strata_2_size = 10**6

        len_sampled = int(0.1*(general_strata_1_size + general_strata_2_size))

        distr_1_general = np.random.normal(100,10,general_strata_1_size) #general distr of strata 1 1
        distr_2_general = np.random.normal(50,5,general_strata_2_size)
        
        total_distr_general = np.concatenate((distr_1_general,distr_2_general)) #general distr of all data
        global_mean = np.mean(total_distr_general)



        indexes = np.random.choice(range(len(total_distr_general)),len_sampled,False)
        distr_1 = total_distr_general[indexes[indexes <= general_strata_1_size]] # distr of sampled data for 1 strata before 
                                                                                    #ab test to compute weights 
        distr_2 = total_distr_general[indexes[indexes > general_strata_1_size]] 

        indexes = np.random.choice(range(len(total_distr_general)),len_sampled,False)
        distr_3 = total_distr_general[indexes[indexes <= general_strata_1_size]] # dist 1-st strate during a/b test  
                                                                                #to sample groups 
        distr_4 = total_distr_general[indexes[indexes > general_strata_1_size]] 


        total_distr = np.concatenate((distr_1,distr_2)) # dist before a/b test to compute weights 
        total_distr_sampled = np.concatenate((distr_3,distr_4)) # distr during ab test to sample data 

        w1 = len(distr_1)/len(total_distr) # we need to compute weights before a/b, otherwise it is overfiriing  
        w2 = len(distr_2)/len(total_distr) # compute and use on the same sampke is not good for generalization 
        se_acc = [] 
   
        indexes = np.random.choice(range(len(total_distr_sampled)),total_len_sampled,False) # samples for a/b
        sample_strat_1 = total_distr_sampled[indexes[indexes <= len(distr_3)]] 
        sample_strat_2 = total_distr_sampled[indexes[indexes > len(distr_3)]]
        var_strat = (np.var(sample_strat_1)*w1 + np.var(sample_strat_2)*w2)/total_len_sampled
        strat_mean = w1 * np.mean(sample_strat_1) + w2 * np.mean(sample_strat_2) # we have a discrepance of var/sqrt(N)
                                                                     #from expectations of general distr 
                                                                     # it will be distinct from  Esrs at var/sqrt(N)
        se_strat = np.sqrt(var_strat)
        se_acc.append(se_strat)
        inside = 1
        if global_mean < strat_mean - 2*se_strat or global_mean > strat_mean + 2*se_strat:
            inside = 0
        contain += inside


    print('ratio within interval strat', contain/n_iters)
    print('mean se strat', np.mean(se_acc))

    means = []
    for _ in range(n_iters*10):
        sampled = np.random.choice(total_distr_sampled,total_len_sampled,True)
        means.append(np.mean(sampled))
    print('se bootstrap',np.std(means))
    

In [79]:
check_strat(1000,100)
check_strat(10000,100)
check_strat(100000,100)
check_strat(200000,100)


<ipython-input-78-52f608137702>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm_notebook(range(n_iters)):



ratio within interval strat 0.93
mean se strat 0.303858499322847
se bootstrap 0.5331388064949822



ratio within interval strat 0.97
mean se strat 0.09560457750249081
se bootstrap 0.16918749227165616



ratio within interval strat 0.94
mean se strat 0.030578171327377633
se bootstrap 0.054637210940334115



ratio within interval strat 0.93
mean se strat 0.021566735742726
se bootstrap 0.037924501455429006
